# ComfyUI (Backend) Server Installation and Configuration Guide


## Summary
This Notebook helps you automate the installation and configuration of ComfyUI, including the following steps:
1. Download ComfyUI: Get the history of versions and select one to download.
2. Set paths: configure extra_model_paths.yaml.
3. Install dependencies: Create a Conda virtual environment and install the required dependencies.
4. Run ComfyUI: Starts the ComfyUI service and saves the logs to the specified location.

Please follow the steps one by one, the system will automatically complete the download, installation and configuration tasks.



## Procedure for Use


### 1. Download ComfyUI
This step will collect all the historical Releases versions of ComfyUI on GitHub, you need to select a version to download.

In [ ]:
import requests

def get_comfyui_versions():
    releases_url = "https://api.github.com/repos/comfyanonymous/ComfyUI/releases"
    response = requests.get(releases_url)
    releases = response.json()
    versions = [release['tag_name'] for release in releases]
    return versions

versions = get_comfyui_versions()
print("  ".join(versions))

Enter the version number you choose, for example, v1.2.3, the system will download the corresponding version of ComfyUI and unzip it to the current directory.

In [ ]:
%%bash

# Modify customized parameters
export COMFYUI_VERSION="v0.3.10"


echo "COMFYUI_VERSION=$COMFYUI_VERSION" >> env_vars.txt

In [ ]:
%%bash

source env_vars.txt

version=$COMFYUI_VERSION
url="https://github.com/comfyanonymous/ComfyUI/archive/refs/tags/$version.tar.gz"
tarball="ComfyUI-$version.tar.gz"

curl -L -o $tarball $url
tar -xzf $tarball

echo "Download and Unzip complete: $tarball"

dirname=$(tar -tf $tarball | head -1 | cut -f1 -d"/")

if [ -d "$dirname" ];then
  mv "$dirname" "ComfyUI"
  echo "The folder has been renamed to: ComfyUI"
  rm $tarball
else
  echo "Can't find the unzipped folder. $dirname"
fi

### 2. Setting the path
This step will modify the ComfyUI official document about the folder loading path and migrate the modified related folder to the specified target location.

Modify the ComfyUI folder path index.

In [ ]:
%%bash

export CURRENT_DIR=$(pwd)

# Modify customized parameters
export COMFYUI_NAME="ComfyUI" # comfyui project name
export TARGET_NAME="autodl-tmp" # Name of the target folder to be migrated


export COMFYUI_DIR="$CURRENT_DIR/$COMFYUI_NAME"
export TARGET_DIR="$CURRENT_DIR/$TARGET_NAME"

echo "ComfyUI path is: $COMFYUI_DIR"
echo "Transfer to the target path as: $TARGET_DIR"

echo "CURRENT_DIR=$CURRENT_DIR" >> env_vars.txt
echo "COMFYUI_NAME=$COMFYUI_NAME" >> env_vars.txt
echo "TARGET_NAME=$TARGET_NAME" >> env_vars.txt
echo "COMFYUI_DIR=$COMFYUI_DIR" >> env_vars.txt
echo "TARGET_DIR=$TARGET_DIR" >> env_vars.txt

In [ ]:
import os

def get_env():
  with open("env_vars.txt", "r") as file:
    for line in file:
      key, value = line.strip().split("=")
      os.environ[key] = value

In [ ]:
get_env()

comfyui_dir = os.getenv('COMFYUI_DIR')
target_name = os.getenv('TARGET_NAME')

file_path = f'{comfyui_dir}/folder_paths.py'
old_line = 'base_path = os.path.dirname(os.path.realpath(__file__))'  # models_dir = os.path.join(base_path, "models")
new_lines = [
    'current_path = os.path.dirname(os.path.realpath(__file__))',
    'parent_path = os.path.dirname(current_path)',
    f'base_path = os.path.join(parent_path, "{target_name}")'
]

with open(file_path, 'r') as file:
    lines = file.readlines()

with open(file_path, 'w') as file:
    for line in lines:
        if old_line in line:
            for new_line in new_lines:
                file.write(new_line + '\n')
        else:
            file.write(line)

print(f"file {file_path} is modification completed")

In [ ]:
%%bash

source env_vars.txt

folders=("custom_nodes" "models")

moved_folders=()
for folder in "${folders[@]}"; do
    if [ -d "$COMFYUI_DIR/$folder" ]; then
        rsync -av --remove-source-files "$COMFYUI_DIR/$folder" "$TARGET_DIR/"
        find "$COMFYUI_DIR/$folder" -type d -empty -delete
        moved_folders+=("$folder")
    else
        echo "$folder folder does not exist"
    fi
done

if [ ${#moved_folders[@]} -eq ${#folders[@]} ]; then
    echo "Movements all complete: ${moved_folders[*]}"
else
    echo "Movements completed: ${moved_folders[*]}"
fi


---

******************  Model Position ⚓  ******************


---





In [ ]:
get_env()

comfyui_dir = os.getenv('COMFYUI_DIR')
target_dir = os.getenv('TARGET_DIR')

os.makedirs(os.path.join(target_dir, "DiM"), exist_ok=True)

content = f"""
other_ui:
   base_path: {target_dir}/DiM
   unet: |
      BlackForestLabs/Model
      TOOLS/ic-light/unet
   checkpoints: |
      Stability/Model
      BlackForestLabs/Model/Flux1/pruned/merge
   style_models: |
      BlackForestLabs/Model/Flux1/inpainting

   vae: |
      TOOLS/vae
   loras: |
      BlackForestLabs/LoRA
      Stability/LoRA
   controlnet: |
      TOOLS/controlnet
   ipadapter: |
      TOOLS/ipadapter

   clip: |
      TOOLS/clip
   clip_vision: |
      TOOLS/clip_vision

   sams: |
      TOOLS/sams
   grounding-dino: |
      TOOLS/grounding-dino

   upscale_models: |
      TOOLS/upscale_models
   rembg: |
      TOOLS/rembg
"""

file_path = os.path.join(comfyui_dir, "extra_model_paths.yaml")
with open(file_path, "w") as file:
    file.write(content)

print(f"extra_model_paths.yaml file has been created and added content to the {file_path}")

### 3. Installation of dependencies
This step creates a virtual environment for ComfyUI to use and installs all dependencies by specifying the image source.

In [ ]:
%%bash

export ENV_NAME="comfyui"
export PYTHON_VERSION="3.12.8"
export PYPI_MIRROR="https://pypi.tuna.tsinghua.edu.cn/simple" # pypi Tsinghua Mirror Source


echo "ENV_NAME=$ENV_NAME" >> env_vars.txt
echo "PYTHON_VERSION=$PYTHON_VERSION" >> env_vars.txt
echo "PYPI_MIRROR=$PYPI_MIRROR" >> env_vars.txt

In [ ]:
%%bash

source env_vars.txt

# conda create -n "$ENV_NAME" python="$PYTHON_VERSION" -y
# echo "Creating a Virtual Environment $ENV_NAME Sucessfully"

# conda activate "$ENV_NAME"

if [ -n "$PYPI_MIRROR" ]; then
    python -m pip config set global.index-url $PYPI_MIRROR
fi

pip install -r "$COMFYUI_DIR/requirements.txt"
echo "Dependencies installed complete"

---

******************  预安装插件 🎯  ******************


---





In [ ]:
%%bash

ADDON_LIST=(
    https://github.com/ltdrdata/ComfyUI-Manager.git # ComfyUI Plugin Management
    https://github.com/ltdrdata/ComfyUI-Impact-Pack.git # Functional node set
)


echo "ADDON_LIST=('$ADDON_LIST[@]')" >> env_vars.txt

In [ ]:
%%bash

source env_vars.txt

cd $TARGET_DIR/custom_nodes

for addon in "${ADDON_LIST[@]}"; do
  git clone "$addon"

  addon_name=$(basename "$addon" .git)

  if [ -f "$addon_name/requirements.txt" ]; then
    pip install -r "$addon_name/requirements.txt"
  fi

  echo "Installation has down: $addon_name"
done

### 4. Run ComfyUI
In the last step, go into the created Conda virtual environment and run ComfyUI. the program will start and listen on port 6006 with low video memory mode (--lowvram) enabled.

In [ ]:
%%bash

export LISTEN_PORT=6006
export EXTRA_OPS=("--lowvram")


echo "LISTEN_PORT=$LISTEN_PORT" >> env_vars.txt
echo "EXTRA_OPS=$EXTRA_OPS" >> env_vars.txt

In [ ]:
%%bash

source env_vars.txt

LOGS_DIR=$COMFYUI_DIR/logs
mkdir -p $LOGS_DIR

echo "LOGS_DIR=$LOGS_DIR" >> env_vars.txt

LOG_FILE="$LOGS_DIR/comfyui.log"

check_and_rename_log() {
    DATE_SUFFIX=$(date +"%Y%m%d_%H%M%S")

    if [ -e "$LOG_FILE" ]; then
        mv "$LOG_FILE" "$LOGS_DIR/comfyui_$DATE_SUFFIX.log"
        echo "The old log file was renamed to $LOGS_DIR/comfyui_$DATE_SUFFIX.log"
    fi
}

start_service() {
    echo "Starting service..."

    check_and_rename_log

    python $COMFYUI_DIR/main.py --listen --port $LISTEN_PORT ${EXTRA_OPS[*]} > "$LOG_FILE" 2>&1 &
    SERVICE_PID=$!

    echo "The service is up and running, PID: $SERVICE_PID"

    if grep -q "SERVICE_PID=" env_vars.txt; then
      sed -i "s/^SERVICE_PID=.*/SERVICE_PID=$SERVICE_PID/" env_vars.txt
    else
      echo "SERVICE_PID=$SERVICE_PID" >> env_vars.txt
    fi
}


start_service



---

---
#### **View Log**


Ongoing surveillance:   `source env_vars.txt && tail -f $LOGS_DIR/comfyui.log`  
Head 50 lines:  `source env_vars.txt && head -n 50 $LOGS_DIR/comfyui.log`  
Tail 50 lines:  `source env_vars.txt && tail -n 50 $LOGS_DIR/comfyui.log`


Turn the page to view:  `source env_vars.txt && less -N +G -20 $LOGS_DIR/comfyui.log`

> `less`    Page through the contents of the document.  
> `-N`  Displays the line number at the beginning of each line.  
> `+G`  Jumps directly to the end of the file.  
> `-20` Displays 20 lines at a time.  

> `Space` or `f`:  Page down.  
> `b`:  Page up.  
> `q`:  Exits less viewing mode.  
> `g`:  Jump to the beginning of the file.  
> `G`:  Jump to the end of the file.  
> `CTRL+D`:  Scroll down half a page.  
> `CTRL+U`:  Scroll up half a page.  

